In [21]:
import numpy as np
import cv2
import os
import random
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
import binascii
list_key=[]
encryptedd_data123=[]
# Function to extract features using ResNet50
def extract_features(image_path):
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features.flatten()

# Function to apply Linear Shift Feedback Register (LSFR)
def lsfr(features):
    # Example LSFR implementation (this can be replaced with your custom implementation)
    random.seed(123)  # Set seed for reproducibility
    lsfr_output = []
    for feature in features:
        bits = format(int(feature), 'b')  # Convert feature to binary string
        lsfr_output.extend(list(bits))  # Append bits to the LSFR output
    return lsfr_output

# Function to generate RSA key pairs from LSFR output
# Function to generate RSA key pairs from LSFR output
# Function to generate RSA key pairs from LSFR output
def generate_keys2(lsfr_output):
    seed = ''.join(lsfr_output)  # Concatenate LSFR output
    seed_bytes = seed.encode('utf-8')  # Convert to bytes
    rng = lambda num_bytes: os.urandom(num_bytes)
    print("random ranges is ",rng)# Use os.urandom for random bytes
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend(),
    )
    public_key = private_key.public_key()
    private_key_length = private_key.key_size  # Access key size from private key object (assuming rsa library supports it)
    public_key_length = public_key.key_size    # Access key size from public key object (assuming rsa library supports it)

    print(f"Private Key Length: {private_key_length} bits")
    print(f"Public Key Length: {public_key_length} bits")
    list_key.append(private_key)
    list_key.append(public_key)
    
    return private_key, public_key

import os
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat, PrivateFormat
from cryptography.exceptions import InvalidSignature

def generate_keys(lsfr_output, min_key_size=2048, max_key_size=4096):
    """
    Generates RSA key pair with a random key length within a specified range.

    Args:
        lsfr_output (str): Output from a Linear Feedback Shift Register (optional).
            If provided, it will be used for seeding the random number generator.
        min_key_size (int, optional): Minimum desired key size in bits. Defaults to 2048.
        max_key_size (int, optional): Maximum desired key size in bits. Defaults to 4096.

    Returns:
        tuple: A tuple containing the generated private key and public key objects.

    Raises:
        ValueError: If min_key_size is less than 1024 or greater than max_key_size.
        ValueError: If max_key_size is less than 1024 or less than min_key_size.
    """

    if min_key_size < 1024 or min_key_size > max_key_size:
        raise ValueError("Minimum key size must be at least 1024 and less than or equal to maximum key size.")

    if max_key_size < 1024:
        raise ValueError("Maximum key size must be at least 1024.")

    # Use a cryptographically secure RNG (CSRNG) based on system randomness
#    if lsfr_output:
        # Consider using a more robust hash function like SHA-256 for seeding
#        seeding_material = hashes.Hash(hashes.SHA256()).update(lsfr_output.encode('utf-8')).finalize()
 #   else:
    seeding_material = os.urandom(32)  # Get 32 bytes of random data

    generator = os.urandom(32)  # Get additional random data for key generation
  #  rng = os.urandom(seeding_material + generator)  # Combine for enhanced randomness

    # Randomly choose a key size within the specified range
    print("extra size",random.getrandbits(max_key_size.bit_length() - min_key_size.bit_length()))
    key_size = min_key_size + random.getrandbits(max_key_size.bit_length() - min_key_size.bit_length())
    
    print("key size ",key_size)

    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=key_size,
        backend=default_backend(),
    )

    public_key = private_key.public_key()

    # Print key lengths for verification (optional)
    print(f"Private Key Length: {private_key.key_size} bits")
    print(f"Public Key Length: {public_key.key_size} bits")

    # Append keys to list_key (if it exists)
    if list_key:  # Check if list_key is defined
        list_key.append(private_key)
        list_key.append(public_key)

    return private_key, public_key
# Function to encrypt data using RSA public key
def encrypt_data(data, public_key):
    encrypted_data = public_key.encrypt(data, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None))
    return encrypted_data

# Function to decrypt data using RSA private key
def decrypt_data(encrypted_data, private_key):
    decrypted_data = private_key.decrypt(encrypted_data, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()), algorithm=hashes.SHA256(), label=None))
    return decrypted_data

# Main function
def main():
    # Step 1: Extract features using ResNet50
    image_path = '2.jpg'
    features = extract_features(image_path)
    
    print("total number of features extracted from each image : ",len(features))

    # Step 2: Apply LSFR
    lsfr_output = lsfr(features)

    # Step 3: Generate RSA key pairs from LSFR output
    private_key, public_key = generate_keys(lsfr_output)

    # Step 4: Encrypt and decrypt data using generated keys
    data_to_encrypt = b"Your data here"
    encrypted_data = encrypt_data(data_to_encrypt, public_key)
    encryptedd_data123.append(encrypted_data)
    
    decrypted_data = decrypt_data(encrypted_data, private_key)

    # Print decrypted data
    print("Original data:", data_to_encrypt)
    print("Decrypted data:", decrypted_data)

#if __name__ == "__main__":
main()


total number of features extracted from each image :  2048
extra size 0
key size  2048
Private Key Length: 2048 bits
Public Key Length: 2048 bits
Original data: b'Your data here'
Decrypted data: b'Your data here'


In [13]:
list_key

In [14]:
# Function to save keys to files
def save_keys(private_key, public_key):
    with open("private_key.pem", "wb") as f:
        f.write(private_key)

    with open("public_key.pem", "wb") as f:
        f.write(public_key)

# Function to load keys from files
def load_keys():
    with open("private_key.pem", "rb") as f:
        private_key = serialization.load_pem_private_key(
            f.read(),
            password=None,
            backend=default_backend()
        )

    with open("public_key.pem", "rb") as f:
        public_key = serialization.load_pem_public_key(
            f.read(),
            backend=default_backend()
        )

    return private_key, public_key

In [15]:
# Serialize the keys

private_key=list_key[0]
public_key=list_key[1]
private_pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

public_pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# Save keys to files
save_keys(private_pem, public_pem)

In [16]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import serialization, hashes
import binascii


# Function to load keys from files
def load_keys():
    with open("private_key.pem", "rb") as f:
        private_key = serialization.load_pem_private_key(
            f.read(),
            password=None,
            backend=default_backend()
        )

    with open("public_key.pem", "rb") as f:
        public_key = serialization.load_pem_public_key(
            f.read(),
            backend=default_backend()
        )

    return private_key, public_key

# Load keys from files
loaded_private_key, loaded_public_key = load_keys()

# Example text to encrypt
#message = b"Hello, this is a test message."



# Decrypt using loaded private key
plaintext = loaded_private_key.decrypt(
    encryptedd_data123[0],
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

#print("Original message:", message.decode())
print("Decrypted message:", plaintext.decode())


Decrypted message: Your data here
